# Bidirectional LSTM + CRF

If you are new to this post, then please refer to [Model Basics](https://github.com/akash1309/Named-Entity-Recognition/blob/master/Model_Basics.ipynb)


Advanced: Making Dynamic Decisions and the Bi-LSTM CRF
======================================================

Dynamic versus Static Deep Learning Toolkits
--------------------------------------------

Pytorch is a *dynamic* neural network kit. Another example of a dynamic
kit is `Dynet <https://github.com/clab/dynet>`__ (I mention this because
working with Pytorch and Dynet is similar. If you see an example in
Dynet, it will probably help you implement it in Pytorch). The opposite
is the *static* tool kit, which includes Theano, Keras, TensorFlow, etc.
The core difference is the following:

* In a static toolkit, you define
  a computation graph once, compile it, and then stream instances to it.
* In a dynamic toolkit, you define a computation graph *for each
  instance*. It is never compiled and is executed on-the-fly

Without a lot of experience, it is difficult to appreciate the
difference. One example is to suppose we want to build a deep
constituent parser. Suppose our model involves roughly the following
steps:

* We build the tree bottom up
* Tag the root nodes (the words of the sentence)
* From there, use a neural network and the embeddings
  of the words to find combinations that form constituents. Whenever you
  form a new constituent, use some sort of technique to get an embedding
  of the constituent. In this case, our network architecture will depend
  completely on the input sentence. In the sentence "The green cat
  scratched the wall", at some point in the model, we will want to combine
  the span $(i,j,r) = (1, 3, \text{NP})$ (that is, an NP constituent
  spans word 1 to word 3, in this case "The green cat").

However, another sentence might be "Somewhere, the big fat cat scratched
the wall". In this sentence, we will want to form the constituent
$(2, 4, NP)$ at some point. The constituents we will want to form
will depend on the instance. If we just compile the computation graph
once, as in a static toolkit, it will be exceptionally difficult or
impossible to program this logic. In a dynamic toolkit though, there
isn't just 1 pre-defined computation graph. There can be a new
computation graph for each instance, so this problem goes away.

Dynamic toolkits also have the advantage of being easier to debug and
the code more closely resembling the host language (by that I mean that
Pytorch and Dynet look more like actual Python code than Keras or
Theano).

Bi-LSTM Conditional Random Field Discussion
-------------------------------------------

For this section, we will see a full, complicated example of a Bi-LSTM
Conditional Random Field for named-entity recognition. The LSTM tagger
above is typically sufficient for part-of-speech tagging, but a sequence
model like the CRF is really essential for strong performance on NER.
Familiarity with CRF's is assumed. Although this name sounds scary, all
the model is is a CRF but where an LSTM provides the features. This is
an advanced model though, far more complicated than any earlier model in
this tutorial. If you want to skip it, that is fine. To see if you're
ready, see if you can:

-  Write the recurrence for the viterbi variable at step i for tag k.
-  Modify the above recurrence to compute the forward variables instead.
-  Modify again the above recurrence to compute the forward variables in
   log-space (hint: log-sum-exp)

If you can do those three things, you should be able to understand the
code below. Recall that the CRF computes a conditional probability. Let
$y$ be a tag sequence and $x$ an input sequence of words.
Then we compute

\begin{align}P(y|x) = \frac{\exp{(\text{Score}(x, y)})}{\sum_{y'} \exp{(\text{Score}(x, y')})}\end{align}

Where the score is determined by defining some log potentials
$\log \psi_i(x,y)$ such that

\begin{align}\text{Score}(x,y) = \sum_i \log \psi_i(x,y)\end{align}

To make the partition function tractable, the potentials must look only
at local features.

In the Bi-LSTM CRF, we define two kinds of potentials: emission and
transition. The emission potential for the word at index $i$ comes
from the hidden state of the Bi-LSTM at timestep $i$. The
transition scores are stored in a $|T|x|T|$ matrix
$\textbf{P}$, where $T$ is the tag set. In my
implementation, $\textbf{P}_{j,k}$ is the score of transitioning
to tag $j$ from tag $k$. So:

\begin{align}\text{Score}(x,y) = \sum_i \log \psi_\text{EMIT}(y_i \rightarrow x_i) + \log \psi_\text{TRANS}(y_{i-1} \rightarrow y_i)\end{align}

\begin{align}= \sum_i h_i[y_i] + \textbf{P}_{y_i, y_{i-1}}\end{align}

where in this second expression, we think of the tags as being assigned
unique non-negative indices.

If the above discussion was too brief, you can check out
`this <http://www.cs.columbia.edu/%7Emcollins/crf.pdf>`__ write up from
Michael Collins on CRFs.

Implementation Notes
--------------------

The example below implements the forward algorithm in log space to
compute the partition function, and the viterbi algorithm to decode.
Backpropagation will compute the gradients automatically for us. We
don't have to do anything by hand.

The implementation is not optimized. If you understand what is going on,
you'll probably quickly see that iterating over the next tag in the
forward algorithm could probably be done in one big operation. I wanted
to code to be more readable. If you want to make the relevant change,
you could probably use this tagger for real tasks.



In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

Helper functions to make the code more readable.

In [0]:
def argmax(vec):
  # return the argmax as a python int
  _, idx = torch.max(vec, 1)
  return idx.item()


def prepare_sequence(seq, to_ix):
  idxs = [to_ix[w] for w in seq]
  return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
  max_score = vec[0, argmax(vec)]
  max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
  return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

Create model

In [0]:
class BiLSTM_CRF(nn.Module):

  def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
    super(BiLSTM_CRF, self).__init__()
    self.embedding_dim = embedding_dim
    self.hidden_dim = hidden_dim
    self.vocab_size = vocab_size
    self.tag_to_ix = tag_to_ix
    self.tagset_size = len(tag_to_ix)

    self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,num_layers=1, bidirectional=True)

    # Maps the output of the LSTM into tag space.
    self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

    # Matrix of transition parameters.  Entry i,j is the score of
    # transitioning *to* i *from* j.
    self.transitions = nn.Parameter(torch.randn(self.tagset_size, self.tagset_size))

    # These two statements enforce the constraint that we never transfer
    # to the start tag and we never transfer from the stop tag
    self.transitions.data[tag_to_ix[START_TAG], :] = -10000
    self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

    self.hidden = self.init_hidden()

  def init_hidden(self):
    return (torch.randn(2, 1, self.hidden_dim // 2),torch.randn(2, 1, self.hidden_dim // 2))

  def _forward_alg(self, feats):
    # Do the forward algorithm to compute the partition function
    init_alphas = torch.full((1, self.tagset_size), -10000.)
    # START_TAG has all of the score.
    init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

    # Wrap in a variable so that we will get automatic backprop
    forward_var = init_alphas

    # Iterate through the sentence
    for feat in feats:
      alphas_t = []  # The forward tensors at this timestep
      for next_tag in range(self.tagset_size):
        # broadcast the emission score: it is the same regardless of
        # the previous tag
        emit_score = feat[next_tag].view(
            1, -1).expand(1, self.tagset_size)
        # the ith entry of trans_score is the score of transitioning to
        # next_tag from i
        trans_score = self.transitions[next_tag].view(1, -1)
        # The ith entry of next_tag_var is the value for the
        # edge (i -> next_tag) before we do log-sum-exp
        next_tag_var = forward_var + trans_score + emit_score
        # The forward variable for this tag is log-sum-exp of all the
        # scores.
        alphas_t.append(log_sum_exp(next_tag_var).view(1))
      forward_var = torch.cat(alphas_t).view(1, -1)
    terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
    alpha = log_sum_exp(terminal_var)
    return alpha

  def _get_lstm_features(self, sentence):
    self.hidden = self.init_hidden()
    embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
    lstm_out, self.hidden = self.lstm(embeds, self.hidden)
    lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
    lstm_feats = self.hidden2tag(lstm_out)
    return lstm_feats

  def _score_sentence(self, feats, tags):
    # Gives the score of a provided tag sequence
    score = torch.zeros(1)
    tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
    for i, feat in enumerate(feats):
      score = score + self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
    score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
    return score

  def _viterbi_decode(self, feats):
    backpointers = []

    # Initialize the viterbi variables in log space
    init_vvars = torch.full((1, self.tagset_size), -10000.)
    init_vvars[0][self.tag_to_ix[START_TAG]] = 0

    # forward_var at step i holds the viterbi variables for step i-1
    forward_var = init_vvars
    for feat in feats:
      bptrs_t = []  # holds the backpointers for this step
      viterbivars_t = []  # holds the viterbi variables for this step

      for next_tag in range(self.tagset_size):
        # next_tag_var[i] holds the viterbi variable for tag i at the
        # previous step, plus the score of transitioning
        # from tag i to next_tag.
        # We don't include the emission scores here because the max
        # does not depend on them (we add them in below)
        next_tag_var = forward_var + self.transitions[next_tag]
        best_tag_id = argmax(next_tag_var)
        bptrs_t.append(best_tag_id)
        viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
      # Now add in the emission scores, and assign forward_var to the set
      # of viterbi variables we just computed
      forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
      backpointers.append(bptrs_t)

    # Transition to STOP_TAG
    terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
    best_tag_id = argmax(terminal_var)
    path_score = terminal_var[0][best_tag_id]

    # Follow the back pointers to decode the best path.
    best_path = [best_tag_id]
    for bptrs_t in reversed(backpointers):
      best_tag_id = bptrs_t[best_tag_id]
      best_path.append(best_tag_id)
    # Pop off the start tag (we dont want to return that to the caller)
    start = best_path.pop()
    assert start == self.tag_to_ix[START_TAG]  # Sanity check
    best_path.reverse()
    return path_score, best_path

  def neg_log_likelihood(self, sentence, tags):
    feats = self._get_lstm_features(sentence)
    forward_score = self._forward_alg(feats)
    gold_score = self._score_sentence(feats, tags)
    return forward_score - gold_score

  def forward(self, sentence):  # dont confuse this with _forward_alg above.
    # Get the emission scores from the BiLSTM
    lstm_feats = self._get_lstm_features(sentence)

    # Find the best path, given the features.
    score, tag_seq = self._viterbi_decode(lstm_feats)
    return score, tag_seq

Run training

In [4]:
# For words

word_filepath = 'drive/My Drive/Pytorch_DataSet/Named Entity Recognition/big/words.txt'
word_to_ix = {}
with open(word_filepath,'r') as f:

  for i,word in enumerate(set(f.read().splitlines())):
    word_to_ix[word] = i   # Because first 2 indices are stored for padding and unknown character

#word_to_ix['<pad>'] = 0  # padding
#word_to_ix['UNK'] = 1    # unknown

ix_to_word = {index: word for word, index in word_to_ix.items()}
print(len(word_to_ix))

35178


In [5]:
# for tags.txt
START_TAG = "<START>"
STOP_TAG = "<STOP>"
tags_filepath = 'drive/My Drive/Pytorch_DataSet/Named Entity Recognition/big/tags.txt'
tag_to_ix = {}

with open(tags_filepath,'r') as f:
  for i,word in enumerate(set(f.read().splitlines())):
    tag_to_ix[word] = i
tag_to_ix[START_TAG] = len(tag_to_ix)
tag_to_ix[STOP_TAG] = len(tag_to_ix)

ix_to_tag = {index: word for word, index in tag_to_ix.items()}
print(len(tag_to_ix))

19


In [6]:

EMBEDDING_DIM = 50
HIDDEN_DIM = 50

# Make up some training data
sent_file = 'drive/My Drive/Pytorch_DataSet/Named Entity Recognition/big/sentences.txt'
label_file = 'drive/My Drive/Pytorch_DataSet/Named Entity Recognition/big/labels.txt'
sentences = []
with open(sent_file,'r') as f:
  txt = f.read().splitlines()

for sent in txt:
  sentences.append(sent)

print(len(sentences))

labels = []
with open(label_file,'r') as f:
  txt = f.read().splitlines()

for lab in txt:
  labels.append(lab)

print(len(labels))
print(sentences[0])
print(labels[0])

data = [(sentences[i].split(), labels[i].split()) for i in range(0, len(sentences))]
# For now lets take only 1000 sentences

training_data = data[:1000]
testing_data = data[1000:1010]
print(len(training_data))
#print(training_set[:2]) 
print(type(training_data))

47959
47959
Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
O O O O O O B-geo O O O O O B-geo O O O O O B-gpe O O O O O
1000
<class 'list'>


In [7]:
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
model

BiLSTM_CRF(
  (word_embeds): Embedding(35178, 50)
  (lstm): LSTM(50, 25, bidirectional=True)
  (hidden2tag): Linear(in_features=50, out_features=19, bias=True)
)

In [8]:
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0
    nesterov: False
    weight_decay: 0.0001
)

In [9]:

# Check predictions before training
with torch.no_grad():
  precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
  precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
  print(model(precheck_sent))


(tensor(56.2492), [9, 14, 13, 9, 14, 13, 9, 14, 13, 9, 14, 13, 9, 14, 13, 9, 14, 13, 9, 14, 13, 9, 3, 7])


In [10]:
losses = []

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(20):  # again, normally you would NOT do 300 epochs, it is toy data
  tracker = 0
  for sentence, tags in training_data:
    tracker += 1
    # Step 1. Remember that Pytorch accumulates gradients.
    # We need to clear them out before each instance
    model.zero_grad()

    # Step 2. Get our inputs ready for the network, that is,
    # turn them into Tensors of word indices.
    sentence_in = prepare_sequence(sentence, word_to_ix)
    targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

    # Step 3. Run our forward pass.
    loss = model.neg_log_likelihood(sentence_in, targets)
    losses.append(loss.item())
    if(tracker%100 == 0):
      print(f'Epochs : {epoch}  Loss Values : {loss.item()}')
    # Step 4. Compute the loss, gradients, and update the parameters by
    # calling optimizer.step()
    loss.backward()
    optimizer.step()



Epochs : 0  Loss Values : 30.58507537841797
Epochs : 0  Loss Values : 3.9963302612304688
Epochs : 0  Loss Values : 11.715789794921875
Epochs : 0  Loss Values : 6.805194854736328
Epochs : 0  Loss Values : 4.2734527587890625
Epochs : 0  Loss Values : 4.63916015625
Epochs : 0  Loss Values : 1.4849205017089844
Epochs : 0  Loss Values : 21.002182006835938
Epochs : 0  Loss Values : 12.66448974609375
Epochs : 0  Loss Values : 0.96624755859375
Epochs : 1  Loss Values : 17.006179809570312
Epochs : 1  Loss Values : 2.584197998046875
Epochs : 1  Loss Values : 6.743438720703125
Epochs : 1  Loss Values : 4.316043853759766
Epochs : 1  Loss Values : 2.0470123291015625
Epochs : 1  Loss Values : 2.385345458984375
Epochs : 1  Loss Values : 1.0657844543457031
Epochs : 1  Loss Values : 12.713905334472656
Epochs : 1  Loss Values : 6.3920135498046875
Epochs : 1  Loss Values : 0.4549407958984375
Epochs : 2  Loss Values : 10.199432373046875
Epochs : 2  Loss Values : 2.389129638671875
Epochs : 2  Loss Values :

In [34]:
# Check predictions after training
with torch.no_grad():
  for sent, tag in testing_data:
    precheck_sent = prepare_sequence(sent, word_to_ix)
    a = model(precheck_sent)
    b = [ix_to_tag[t] for t in a[1]]
    print(f'Predicted Labels : {b}')
    print(f'Actual Labels :    {tag}')
    print('\n\n')  

Predicted Labels : ['O', 'B-per', 'I-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Actual Labels :    ['O', 'O', 'O', 'O', 'O', 'O', 'B-org', 'O', 'O', 'O', 'O', 'O']



Predicted Labels : ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Actual Labels :    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']



Predicted Labels : ['B-per', 'I-per', 'I-per', 'I-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Actual Labels :    ['O', 'O', 'B-per', 'I-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']



Predicted Labels : ['O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Actual Labels :    ['O', 'O', 'B-org', 'O', 'O', 'O', 'O', 'O', 'O', 'O']



Predicted Labels : ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-org', 'I-org', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Actual Labels :    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-org', 'I-org', 'I-org', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

Model Efficiency can be increased with hyperparameter tuning.